<h1 align="center"> 海洋数据分析 第九讲</h1>
<h3 align="right">主讲人: 陈笔澄</h3>
<h2>目录<span class="tocSkip"></span></h2>

- ## [数据格式--NetCDF](#sec:netcdf)
- ## [添加地图--Cartopy](#sec:map)
- ## [一个更复杂的例子](#sec:example)

## 帮助链接:
- ## NetCDF4: https://unidata.github.io/netcdf4-python/
- ## Cartopy: https://scitools.org.uk/cartopy/docs/latest/
## 目标:
- ## 读取ERA5的海表面温度数据(Sea Surface Temperature, SST)，并利用Python将数据展示于世界地图之上。
### 🔍 注: ERA5是欧洲中期天气预报中心(ECMWF)的全球小时再分析数据(https://www.ecmwf.int/en/forecasts/datasets/reanalysis-datasets/era5)

---
---

## 

---
---

## 🗂 <a id = "sec:netcdf"> 数据格式NetCDF </a>

### 🔍 注: 在将来的科研及工作中，大家会接触到各种各样的数据格式(e.g., CSV, Grib, Grib2, NetCDF):
- ### 幸运的是，一般Python都会有相应的模块对数据进行基本的读取与处理操作。
- ### 作为用户，我们不需要像过去使用Python基本文件读写功能那样注意具体的读取和处理细节，只需要掌握相应的函数及参数即可。

---

## 💡 读取NetCDF格式的ERA5数据

In [ ]:
from netCDF4 import Dataset

# 读取数据
fn_era5 = "./attachment/era5_SST.nc"
ncid = Dataset(fn_era5, "r")

In [ ]:
# 观察数据集
ncid.variables

# 变量说明包含变量类型、变量名、单位、完整名称、维度信息、数据说明

In [ ]:
## 观察我们要画的SST(Sea Surface Temperature)变量
ncid.variables["sst"]

In [ ]:
## 取数据，需要用到“键”(key)以及[:]运算符
latitude = ncid["latitude"][:]
longitude = ncid["longitude"][:]
sst = ncid["sst"][:]

---

## 💡 先采用我们之前学过的知识做成果展示

In [ ]:
## 使用matplotlib画图
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# 画布
fig, ax = plt.subplots()

# 画图
cmap_sst = cm.get_cmap('viridis') # 选择颜色图
levels_sst = np.linspace(270, 304, 18) # 设置等值线
cax = ax.contourf(longitude, latitude, sst[0, :, :], levels=levels_sst, cmap=cmap_sst, extend='both')

# 说明
ax.set_xlabel("Longitude (DEG)")
ax.set_ylabel("Latitude (DEG)")

# 展示
plt.show()

## ❓ 我们要怎么添加地图
## ❓ 我们要怎么采用不同的投影（e.g., 比如采用地球的真实比例）

---
---

## 

---
---

## 🗂 <a id = "sec:netcdf"> 添加地图--Cartopy </a>

---

## ❓ 在使用Cartopy作图是，需要在坐标中指定相应投影

## 💡 `cartopy.crs`子模块(coordinate reference system)

In [ ]:
## 模块
import cartopy.crs as ccrs # coordinate reference system
import matplotlib.pyplot as plt

In [ ]:
# 画布
ax = plt.axes(projection=ccrs.PlateCarree()) # 投影

ax.coastlines() # 海岸线

In [ ]:
## 试试不同的投影--真实地球比例
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=100, central_latitude=35))
ax.coastlines(resolution='50m') # 海岸线

In [ ]:
## 试试不同的投影--椭圆体(摩尔维特投影)
ax = plt.axes(projection=ccrs.Mollweide())
ax.coastlines() # 海岸线

### 🔍 关于投影，可查阅 https://scitools.org.uk/cartopy/docs/v0.15/crs/projections.html

---

## ❓ 如何添加其他地图元素(e.g., 国界线)
## 💡 使用`feature`子模块与`ax.add_feature()`子函数

In [ ]:
import cartopy.feature as cfeature

In [ ]:
ax = plt.axes(projection=ccrs.Mollweide())
ax.coastlines() # 海岸线
ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.5) # 国界线
plt.show()

## 

## 💡 使用`feature`子模块中的`NaturalEarthFeature()`子函数填充陆地效果

In [ ]:
ax = plt.axes(projection=ccrs.Mollweide())
ax.coastlines() # 海岸线
ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.5, zorder=2) # 国界线
land_50m = cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='face',
                                        facecolor=cfeature.COLORS['land_alt1'])
ax.add_feature(land_50m, zorder=1) # 陆地效果
plt.show()

## 🔍 更多关于`cartopy.feature`: https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html

---

## 😱 填充数据完成成果展示

In [ ]:
ax = plt.axes(projection=ccrs.Mollweide())

# SST
cax = ax.contourf(longitude, latitude, sst[0, :, :],
    levels=levels_sst, cmap=cmap_sst, extend='both')

ax.coastlines() # 海岸线
ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.5, zorder=2) # 国界线
land_50m = cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='face',
                                        facecolor=cfeature.COLORS['land_alt1'])
ax.add_feature(land_50m, zorder=1) # 陆地效果
plt.show()

## ❓ 怎么不对
## 💡 没有告诉地图，我们的数据采用什么坐标体系，运用`transform`参数解决问题

In [ ]:
ax = plt.axes(projection=ccrs.Mollweide())

# SST
cax = ax.contourf(longitude, latitude, sst[0, :, :],
    levels=levels_sst, cmap=cmap_sst, extend='both',
    transform=ccrs.PlateCarree()) ## transform参数指定的是数据的坐标投影，而不是画本身的坐标投影

ax.coastlines() # 海岸线
ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.5, zorder=2) # 国界线
land_50m = cfeature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='face',
                                        facecolor=cfeature.COLORS['land_alt1'])
ax.add_feature(land_50m, zorder=1) # 陆地效果
plt.show()

## 🔍 注: `transform`参数指定的是数据的坐标投影，而不是画本身的坐标投影。

---
---

## 

---
---

## 🗂 <a id = "sec:example"> 一个更复杂的例子</a>

## 🔍 请注意日期如何从小时转化为完整的日期格式

In [ ]:
## 载入模块
from netCDF4 import Dataset
import numpy as np
from datetime import datetime, timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MultipleLocator
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
## 自定义参数
# 时间
time_anchor = datetime(1900, 1, 1, 0, 0, 0) #(year, month, day, hour, minitue, second)

# 图片
figsize = (8, 4)
ncol_fig = 2
levels_sst = np.linspace(270, 304, 18)
cmap_sst = cm.get_cmap('viridis')
param_gs = dict(left=0.05, right=0.92, bottom=0.05, top=0.98, hspace=0.2, wspace=0.1) # parameters for gridspec
lat_spacing = 30
lon_spacing = 30

In [ ]:
## 读取数据
fn_era5 = "./attachment/era5_SST.nc"
ncid = Dataset(fn_era5, "r")

time = ncid["time"][:] # hours since 1900-01-01 00:00:00.0
latitude = ncid["latitude"][:]
longitude = ncid["longitude"][:]
sst = ncid["sst"][:]

In [ ]:
## 数据处理
# 计算时间维度
nt = len(time)

# 将时间转化为年月日时分秒模式
time_full = np.array([timedelta(hours=int(hour)) + time_anchor for hour in time])
print("The time for data:\n", time_full)

In [ ]:
## 成果展示
# 预处理
land_50m = cfeature.NaturalEarthFeature('physical', 'land', '50m',
    edgecolor='face', facecolor=cfeature.COLORS['land_alt1']) # 陆地颜色设置

## 构图
fig = plt.figure(figsize=figsize)
gs = fig.add_gridspec(int(np.ceil(nt/ncol_fig)), ncol_fig, **param_gs)

## 画图
for isub in range(nt):
    # 海表面温度
    ax = fig.add_subplot(gs[isub], projection=ccrs.Mollweide())
    cax = ax.contourf(longitude, latitude, sst[isub, :, :],
        levels=levels_sst, cmap=cmap_sst, extend='both',
        transform=ccrs.PlateCarree())
    
    # 辅助效果
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=0.5, zorder=2) # 国界线
    ax.add_feature(land_50m, zorder=1) # 陆地效果
    
    ## 说明
    gl=ax.gridlines(crs=ccrs.PlateCarree(),
        linewidth=1, color='gray', alpha=0.5, linestyle='--') # 经纬线(map gridline object)
    gl.xlocator = MultipleLocator(lon_spacing)
    gl.ylocator = MultipleLocator(lat_spacing)
    
    ax.set_title(time_full[isub].strftime("%Y/%m/%d %H:%M UTC"))

# 添加色阶图
cbar_ax = plt.axes([0.92, 0.32, 0.02, 0.3]) # 建立一个新的子区域，格式是[x起始, y起始, 宽度, 高度]，单位是整个画布的归一化坐标。
cbar = fig.colorbar(cax, cax=cbar_ax, ticks=levels_sst[::3])
cbar.ax.set_title("T $(^\circ C)$", ha='center', va='bottom')
    
## 展示
plt.show()

---
---